In [5]:
!pip install xgboost
import numpy as np
import os
import plotly as py
import plotly.io as pio
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from warnings import simplefilter
simplefilter(action='ignore', category=Warning)
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)

data = pd.read_csv('AAPL.csv')

df = data.copy()
data = data.iloc[9344:]
data.head(5)

# AAPL Stock Prices from 2018 to 2023
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.Date, y=data.Close))
fig.update_layout(title='AAPL Stock Prices from 2018 to 2023')
fig.show()

# AAPL Stock Prices and Volume from 2018 to 2023
fig = make_subplots(rows=3,cols=1)
fig.add_trace(go.Ohlc(x=data.Date,open=data.Open,high=data.High,low=data.Low,close=data.Close,name='Price'),row=1,col=1)
fig.add_trace(go.Scatter(x=data.Date, y=data.Volume, name='Volume'),row=3,col=1)
fig.update(layout_xaxis_rangeslider_visible=False)
fig.update_layout(title='AAPL Stock Prices and Volume from 2018 to 2023')
fig.show()

# Splitting AAPL Stock data
testsize = 0.10
validsize = 0.10
test_split_idx = int(data.shape[0]*(1-testsize))
valid_split_idx = int(data.shape[0]*(1-(validsize+testsize)))
train_data = data.iloc[:valid_split_idx].copy()
valid_data = data.iloc[valid_split_idx+1:test_split_idx].copy()
test_data = data.iloc[test_split_idx+1:].copy()
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_data.Date, y=train_data.Close, name='Training'))
fig.add_trace(go.Scatter(x=valid_data.Date, y=valid_data.Close, name='Validation'))
fig.add_trace(go.Scatter(x=test_data.Date,  y=test_data.Close,  name='Test'))
fig.update_layout(title='Splitting AAPL Stock data')
fig.show()

drop_columns = ['Date', 'Volume', 'Open', 'Low', 'High']
train_data = train_data.drop(columns=drop_columns)
valid_data = valid_data.drop(columns=drop_columns)
test_data = test_data.drop(columns=drop_columns)

X_train = train_data.drop(columns='Close')
y_train = train_data['Close'].copy()
X_valid = valid_data.drop(columns='Close')
y_valid = valid_data['Close'].copy()
X_test = test_data.drop(columns='Close')
y_test = test_data['Close'].copy()
print(X_train)
print(y_train)

evaluation = [(X_train, y_train), (X_valid, y_valid)]

params = {
    'objective': 'reg:squarederror',  # Specify the regression objective
    'max_depth': 8,                  # Maximum depth of the tree
    'learning_rate': 0.2,           # Learning rate
    'n_estimators': 100,             # Number of boosting rounds
    'gamma': 0.005,               # Fraction of samples used for training
    'random_state': 42,           # Fraction of features used for training
    "enable_categorical": True,
}

model = xgb.XGBRegressor(**params)
model.fit(X_train, y_train, eval_set=evaluation, verbose=False)

# mse
y_pred = model.predict(X_test)
print(f'y_true = {np.array(y_test)[:5]}')
print(f'y_pred = {y_pred[:5]}')
print(f'mean_squared_error = {mean_squared_error(y_test, y_pred)}')

# r2 score
r2_score(y_test,y_pred)

predicted_prices = data.iloc[test_split_idx+1:].copy()
predicted_prices['Close'] = y_pred

fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=data.Date, y=data.Close,
                         name='Real Value',
                         marker_color='Red'), row=1, col=1)

fig.add_trace(go.Scatter(x=predicted_prices.Date,
                         y=predicted_prices.Close,
                         name='Prediction',
                         marker_color='Green'), row=1, col=1)

fig.add_trace(go.Scatter(x=predicted_prices.Date,
                         y=y_test,
                         name='Real Value',
                         marker_color='Red',
                         showlegend=False), row=2, col=1)

fig.add_trace(go.Scatter(x=predicted_prices.Date,
                         y=y_pred,
                         name='Prediction',
                         marker_color='Green',
                         showlegend=False), row=2, col=1)
fig.update_layout(title='AAPL Stock Price Prediction using XGBoost')

fig.show()

       Day Difference   Adj Close
9344          0.52500   40.776527
9345         -0.07500   40.769421
9346          0.12250   40.958794
9347          0.39000   41.425125
9348          0.00000   41.271263
...               ...         ...
10496         0.50000  160.334778
10497        -0.09001  158.845688
10498         5.29001  164.921188
10499        -0.20000  164.603516
10500         2.14000  164.374878

[1157 rows x 2 columns]
9344      43.064999
9345      43.057499
9346      43.257500
9347      43.750000
9348      43.587502
            ...    
10496    161.509995
10497    160.009995
10498    166.130005
10499    165.809998
10500    165.350006
Name: Close, Length: 1157, dtype: float64
y_true = [152.869995 150.589996 148.5      150.470001 152.589996]
y_pred = [153.9624  151.44342 149.5636  151.63647 153.6311 ]
mean_squared_error = 34.27132010537187
